In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import os, time

np.random.seed(2)

In [ ]:
# Generating data
T = 20
L = 1000
N = 100

x = np.empty((N, L), 'int64')
x[:] = np.array(range(L)) + np.random.randint(-4 * T, 4 * T, N).reshape(N, 1)
data = np.sin(x / 1.0 / T).astype('float64')
torch.save(data, open('traindata.pt', 'wb'))

# Number of steps for training
steps = 1

In [ ]:
class Sequence(nn.Module):
    def __init__(self, device):
        super(Sequence, self).__init__()
        # creating cells lstm1, lstm2, linear layer for prediction
        self.lstm1 = nn.LSTMCell(1, 51)
        self.lstm2 = nn.LSTMCell(51, 51)
        self.linear = nn.Linear(51, 1)
        self.device = device
      # we defining forward propagation 
    def forward(self, input, future = 0):
        outputs = []
        h_t = torch.zeros(input.size(0), 51, dtype=torch.double, device=self.device)
        c_t = torch.zeros(input.size(0), 51, dtype=torch.double, device=self.device)
        h_t2 = torch.zeros(input.size(0), 51, dtype=torch.double, device=self.device)
        c_t2 = torch.zeros(input.size(0), 51, dtype=torch.double, device=self.device)
       # it takes input
        for input_t in input.split(1, dim=1):
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
           # it gives us output
        for i in range(future):# if we should predict the future
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        outputs = torch.cat(outputs, dim=1)
        return outputs

In [ ]:
def run_on(steps, device):
  # build the model
  seq = Sequence(device)
  seq.double()
  criterion = nn.MSELoss()
  # use LBFGS as optimizer since we can load the whole data to train
  optimizer = optim.LBFGS(seq.parameters(), lr=0.8)
  # export model to corresponding device
  seq.to(device)

  # set random seed to 0
  np.random.seed(0)
  torch.manual_seed(0)
  # load data and make training set
  data = torch.load('traindata.pt')
  input = torch.from_numpy(data[3:, :-1])
  target = torch.from_numpy(data[3:, 1:])
  test_input = torch.from_numpy(data[:3, :-1])
  test_target = torch.from_numpy(data[:3, 1:])
  # exporting the variables to specific device memory
  input = input.to(device)
  target = target.to(device)
  test_input = test_input.to(device)
  test_target = test_target.to(device)

  # create directories to store the predictions
  os.makedirs(device, exist_ok=True)

  #begin initializing to train
  time_taken = 0
  train_losses = []
  test_losses = []
  for i in range(steps):
      # print('STEP: ', i)
      def closure():
          optimizer.zero_grad()
          out = seq(input)
          loss = criterion(out, target)
          # print('loss:', loss.item())
          train_losses.append(loss.item())
          loss.backward()
          return loss
      optimizer.step(closure)
      # optimizer.step()
      # print("optdone")
      # begin to predict, no need to track gradient here
      tic = time.time()
      with torch.no_grad():
          future = 1000
          pred = seq(test_input, future=future)
          loss = criterion(pred[:, :-future], test_target)
          # print('test loss:', loss.item())
          test_losses.append(loss.item())
          y = pred.detach().cpu().numpy()
      toc = time.time()
      time_taken += toc - tic
      # draw the result
      plt.figure(figsize=(30,10))
      plt.title('Predict future values for time sequences\n(Dashlines are predicted values)', fontsize=30)
      plt.xlabel('x', fontsize=20)
      plt.ylabel('y', fontsize=20)
      plt.xticks(fontsize=20)
      plt.yticks(fontsize=20)
      def draw(yi, color):
          plt.plot(np.arange(input.size(1)), yi[:input.size(1)], color, linewidth = 2.0)
          plt.plot(np.arange(input.size(1), input.size(1) + future), yi[input.size(1):], color + ':', linewidth = 2.0)
      draw(y[0], 'r')
      draw(y[1], 'g')
      draw(y[2], 'b')
      plt.savefig('./{}/predict{}.png'.format(device, i))
      plt.close()

  time_taken = time_taken/steps

  return time_taken, train_losses, test_losses

In [ ]:
# Training on CPU
time_take_cpu, train_losses_cpu, test_losses_cpu = run_on(steps, "cpu")
print("Time taken on CPU: {}s".format(np.round(time_take_cpu, 3)))
# Training on GPU
time_take_gpu, train_losses_gpu, test_losses_gpu = run_on(steps, "cuda:0")
print("Time taken on GPU: {}s".format(np.round(time_take_gpu, 3)))

Time taken on CPU: 0.267s
Time taken on GPU: 0.308s


In [ ]:
# Plotting CPU vs GPU training loss
plt.figure()
plt.title('Comparing Training Loss between CPU and GPU', fontsize=30)
plt.xlabel('#Iterations', fontsize=20)
plt.ylabel('Loss', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.plot(train_losses_cpu, label="CPU")
plt.plot(train_losses_gpu, label="GPU")
plt.legend()
plt.savefig('train_comparision.pdf', bbox_inches='tight')
plt.close()